In [ ]:
import os
import numpy as np
import pandas as pd
import cv2

from tqdm import tqdm
from joblib import Parallel, delayed

In [ ]:
WIDTH = 256
HEIGHT = 256

In [ ]:
data_folder = "/kaggle/input/hotel-id-to-combat-human-trafficking-2022-fgvc9/"
train_folder = os.path.join(data_folder, 'train_images')
hotel_ids = os.listdir(train_folder)

print(len(hotel_ids))

In [ ]:
train_df = pd.DataFrame(columns={'image_id', 'hotel_id'})
for hotel_id in tqdm(hotel_ids):
    for image_id in os.listdir(os.path.join(train_folder, hotel_id)):
        train_df = train_df.append({'image_id': image_id, 'hotel_id': hotel_id}, ignore_index=True)

In [ ]:
train_df

In [ ]:
def pad_image(img):
    w, h, c = np.shape(img)
    if w > h:
        pad = int((w - h) / 2)
        img = cv2.copyMakeBorder(img, 0, 0, pad, pad, cv2.BORDER_CONSTANT, value=0)
    else:
        pad = int((h - w) / 2)
        img = cv2.copyMakeBorder(img, pad, pad, 0, 0, cv2.BORDER_CONSTANT, value=0)
        
    return img


def open_and_preprocess_image(image_folder, image_name, PAD=True):
    img = cv2.imread(os.path.join(image_folder, image_name))
    
    if PAD:
        img = pad_image(img)
    
    return cv2.resize(img, (WIDTH, HEIGHT))


def save_image(image_name, img):
    cv2.imwrite(image_name, img)
    
    
def process_image(data_folder, hotel_id, PAD=True):
    hotel_folder = os.path.join(data_folder, hotel_id)
    
    for image_name in os.listdir(hotel_folder):
        img = open_and_preprocess_image(hotel_folder, image_name, PAD)
        save_image(image_name, img)

example

The 5th room of the 1st hotel room

In [ ]:
i = os.listdir(os.path.join(train_folder,hotel_ids[0]))[4]

In [ ]:
i

In [ ]:
image = os.path.join(os.path.join(train_folder,hotel_ids[0]) , i)
img_array = cv2.imread(image)
img_array.shape

In [ ]:
from matplotlib import pyplot as plt

plt.imshow(img_array)
plt.show()

In [ ]:
try_to_resize = cv2.resize(img_array, (WIDTH, HEIGHT))

In [ ]:
plt.imshow(try_to_resize)
plt.show()

In [ ]:
img_pad_array = pad_image(img_array)
img_pad_array.shape

In [ ]:
resize_img = cv2.resize(img_pad_array, (WIDTH, HEIGHT))
resize_img.shape

In [ ]:
plt.imshow(resize_img)
plt.show()

Data processing with padding

In [ ]:
!pwd

In [ ]:
%%time
dfs_proc = Parallel(n_jobs=4, prefer='threads')(delayed(process_image)(train_folder, hotel_ids[i], PAD=True) for i in range(0, len(hotel_ids)))

In [ ]:
!cd /kaggle/working/ & zip -jqr images.zip .
!find . -name "*.jpg" -delete
train_df.to_csv('train_padding_256.csv', index=False)